reference https://www.kaggle.com/gaoyuan19930220/xgboost

In [2]:
import numpy as np
import pandas as pd
import os
import sklearn
import xgboost as xgb
import operator
import optuna

from functools import partial
from matplotlib import pylab as plt
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold


import warnings
warnings.filterwarnings('ignore')
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [3]:
count_itr = 0

In [4]:
def objective_optuna(X, y, trial):
    global count_itr
    print(count_itr, end=' ')
    count_itr += 1
    #最適化するパラメータを指定
    params = {
        'n_estimators': trial.suggest_int('n_estimators',1,1000),
        'gamma': trial.suggest_uniform('gamma',0,1),
        'reg_lambda': trial.suggest_uniform('lambda',0,2),
        'learning_rate': trial.suggest_uniform('learning_rate',0,0.3),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree',0,1),
        'min_child_weight': trial.suggest_uniform('min_child_weight',0,10),
    }
    
    #モデルを定義
    model = xgb.XGBClassifier(random_state=2,**params)
    
#     (train_X, test_X ,train_y, test_y) = train_test_split(X, y, test_size = 0.3, random_state = 2)
    
#     #3-foldクロスバリデーション
    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=2)
    scores = cross_validate(model, X=X, y=y, cv=kf, scoring='accuracy')
    
    return 1.0 - scores['test_score'].mean()

In [5]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [6]:
def preproceed(dataframe):
    # 欠損値の処理
    dataframe.replace({'Nan':pd.np.nan,'nan':pd.np.nan},inplace=True)
#     dataframe.fillna(dataframe.mean(numeric_only=True),inplace=True)
#     dataframe['Fare'].fillna(dataframe['Fare'].mean(),inplace=True)
#     dataframe['Embarked'].fillna(dataframe['Embarked'].mode().loc[0],inplace=True)
    
    #自分以外の家族の人数及び有無を追加
    dataframe['NumFamily'] = dataframe['SibSp'] + dataframe['Parch']
    dataframe['IsFamily'] = dataframe['NumFamily'] != 0

    #質的変数をダミー変数に
    dataframe['IsPclass1'] = dataframe['Pclass'] == 1
    dataframe['IsPclass2'] = dataframe['Pclass'] == 2
    dataframe['IsPclass2'] = dataframe['Pclass'] == 3
    dataframe['IsEmbarkedS'] = dataframe['Embarked'] == 'S'
    dataframe['IsEmbarkedC'] = dataframe['Embarked'] == 'C'
    dataframe['IsEmbarkedQ'] = dataframe['Embarked'] == 'Q'
    
    
    #質的変数を変換'
    dataframe['Sex'] = dataframe['Sex'].apply(lambda x: 1 if x == 'male' else 0)
#     dataframe['Embarked'] = dataframe['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2}).astype('int')


    #いらない列(Cabin,Name,PassengerId,Ticket)の削除
    dataframe.drop(['Cabin','Name','Ticket','Embarked','Pclass'],axis=1,inplace=True)

In [7]:
#csvデータの呼び出し場所
loadpath = "../input/"
#csvデータの保存場所
savepath = "../output/"

In [8]:
df = pd.read_csv(loadpath + "train.csv")
df_test = pd.read_csv(loadpath + "test.csv")

df.set_index('PassengerId',inplace=True)
df_test.set_index('PassengerId',inplace=True)


train_index = df.index
test_index = df_test.index

preproceed(df)
preproceed(df_test)

In [14]:
df.Sex

PassengerId
1      1
2      0
3      0
4      0
5      1
6      1
7      1
8      1
9      0
10     0
11     0
12     0
13     1
14     1
15     0
16     0
17     1
18     1
19     0
20     0
21     1
22     1
23     0
24     1
25     0
26     0
27     1
28     1
29     0
30     1
      ..
862    1
863    0
864    0
865    1
866    0
867    0
868    1
869    1
870    1
871    1
872    0
873    1
874    1
875    0
876    0
877    1
878    1
879    1
880    0
881    0
882    1
883    0
884    1
885    1
886    0
887    1
888    0
889    0
890    1
891    1
Name: Sex, Length: 891, dtype: int64

In [ ]:
train = df.drop('Survived', axis=1)
labels = df.Survived
test = df_test
#データを分割
#(train_X, test_X ,train_y, test_y) = train_test_split(train_X, train_y, test_size = 0.3, random_state = 666)

In [58]:
train = train.astype('float')

In [10]:
xgboost.callback.record_evaluation333

NameError: name 'xgboost' is not defined

optunaでハイパーパラメーター決定

In [13]:
f = partial(objective_optuna, train, labels)
study = optuna.create_study()
study.optimize(f, n_trials=10000)

0 

[I 2019-02-10 03:45:13,388] Finished a trial resulted in value: 0.20864300755605092. Current best value is 0.20864300755605092 with parameters: {'n_estimators': 489, 'gamma': 0.012190267344302064, 'lambda': 1.3810873697095867, 'learning_rate': 0.21265148591217936, 'max_depth': 12, 'subsample': 0.6857036969434205, 'colsample_bytree': 0.7691871226120469, 'min_child_weight': 8.119943708041982}.


1 

[I 2019-02-10 03:45:13,484] Finished a trial resulted in value: 0.19740183327139837. Current best value is 0.19740183327139837 with parameters: {'n_estimators': 66, 'gamma': 0.800882651880607, 'lambda': 0.6443323829305605, 'learning_rate': 0.2765288102769204, 'max_depth': 3, 'subsample': 0.9003060457019632, 'colsample_bytree': 0.5150371827292576, 'min_child_weight': 0.888889196878494}.


2 

...

[I 2019-02-10 03:47:23,811] Finished a trial resulted in value: 0.21339650687476774. Current best value is 0.18457357859531776 with parameters: {'n_estimators': 399, 'gamma': 0.8943521746894877, 'lambda': 0.6101970248867601, 'learning_rate': 0.1192141873022246, 'max_depth': 12, 'subsample': 0.9217200574951616, 'colsample_bytree': 0.22507627225236979, 'min_child_weight': 0.029946137134566383}.


213 

[I 2019-02-10 03:47:24,643] Finished a trial resulted in value: 0.21184039390561127. Current best value is 0.18457357859531776 with parameters: {'n_estimators': 399, 'gamma': 0.8943521746894877, 'lambda': 0.6101970248867601, 'learning_rate': 0.1192141873022246, 'max_depth': 12, 'subsample': 0.9217200574951616, 'colsample_bytree': 0.22507627225236979, 'min_child_weight': 0.029946137134566383}.


214 

KeyboardInterrupt: 

In [14]:
# train a xgboost classifier
# params = {"num_class": 2}
# train_xgb = xgb.DMatrix(train, labels)
# test_xgb  = xgb.DMatrix(test)
# trainRound = 100
# gbm = xgb.train(params, train_xgb, trainRound)
# pred = gbm.predict(test_xgb)
# pred = pred.astype('int')

In [36]:
study.best_params

{'n_estimators': 399,
 'gamma': 0.8943521746894877,
 'lambda': 0.6101970248867601,
 'learning_rate': 0.1192141873022246,
 'max_depth': 12,
 'subsample': 0.9217200574951616,
 'colsample_bytree': 0.22507627225236979,
 'min_child_weight': 0.029946137134566383}

In [60]:
kf = StratifiedKFold(n_splits=10, shuffle=True,)
scores = cross_validate(model, X=train, y=labels, cv=kf, scoring='accuracy')
scores['test_score'].mean()

0.817162637612076

In [46]:
#最適化するパラメータを設定
    
#モデルの定義
model = xgb.XGBClassifier(random_state=2,**study.best_params)
model.fit(train, labels,eval_metric='error')

In [61]:
pred = model.predict(test)

In [62]:
# create submission file
submission_result = pd.DataFrame({"Survived":pred},index=test.index)
submission_result.to_csv(savepath + 'submission_xgb.csv',index='False')

# 以下適当なコード

In [ ]:
#重要度のやつ


features = train.columns
# create a feature map
outfile = open('xgb.fmap', 'w')
i = 0
for feat in features:
    outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    i = i + 1
outfile.close()

# plot feature importance
importance = gbm.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))
df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()
plt.figure()
df.plot()
df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 20))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
plt.gcf().savefig('feature_importance_xgb.png')



In [31]:

#使うデータの前処理
train_xgb = xgb.DMatrix(df.drop('Survived', axis=1), df.Survived)
test_xgb  = xgb.DMatrix(df_test)
    
#最適化するパラメータを設定
params = {
     'n_estimators': 100,
     'gamma': 0,
     'reg_lambda': 1,
      #'max_depth': trial.suggest_int('max_depth', 2, 32)
}
    
#モデルの定義
model = xgb.XGBClassifier(random_state=2,**params)
model.fit(df.drop('Survived', axis=1), df.Survived)
    
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
# scores = cross_validate(model, X=X, y=y, cv=kf, scoring='accuracy')
    
# return 1.0 - scores['test_score'].mean()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=2,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)